# Registering a TF/Keras (EfficientNet ImageNet Classification) model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to catalog a Keras model on Verta as a Verta Standard Model by convinience functions.

Updated for Verta version: 0.21.0

This notebook walks through initializing EfficientNet Classification model through keras applications, and cataloging them to the Verta platform.

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/registry_examples/registry/tensorflow/keras-imagenet-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Imports

In [ ]:
!python -m pip install verta
!python -m pip install tensorflow
!python -m pip install wget

In [ ]:
import tensorflow as tf
import numpy as np

## 1. Register model

### 1.1 Define Model

A model has to exist before we can register, so we will instantiate one here in our notebook.

**Model Info**

EfficientNetV2 are a family of image classification models, which achieve better parameter efficiency and faster training speed than prior arts. Built upon EfficientNetV1, EfficientNetV2 models use neural architecture search (NAS) to jointly optimize model size and training speed, and are scaled up in a way for faster training and inference speed.

Reference Paper: [EfficientNetV2: Smaller Models and Faster Training](https://arxiv.org/abs/2104.00298) Git Repo: [EfficientNetV2 TensorFlow Repo](https://github.com/google/automl/tree/master/efficientnetv2)


In [ ]:
model = tf.keras.applications.EfficientNetV2M(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
)

As a sanity check, we can validate that our model can produce predictions.

In [ ]:
import os
import wget

# Step 1: Load Image
img_path = "n02106662_German_shepherd.JPEG"
if not os.path.exists(img_path):
    wget.download("https://raw.githubusercontent.com/EliSchwartz/imagenet-sample-images/master/n02106662_German_shepherd.JPEG")
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(480, 480))

# Step 2: Preprocess the image
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# Step 3: Use the model and print the predicted category
print(tf.keras.applications.efficientnet_v2.decode_predictions(model.predict(x, verbose=0), top=3))

### 1.2. Register Model to Verta Model Catalog

Now that the model is in a good shape, we can register it into the Verta platform.

We'll connect to Verta through the [Verta Python Client](https://verta.readthedocs.io/en/main/_autogen/verta.Client.html)
create a [registered model](https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModel.html) for our EfficientNet ImageNet Classification Model  
and a [version](https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModelVersion.html) to associate this particular model with.

All of these can be viewed in the Verta web app once they are created.

Note: If your model uses CustomObject you have to register the model into the catalog using a serialized version of the model by extending the [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase) class

In [ ]:
# Paste your credentials in this cell or anywhere above this along with the code snippet to connect to Verta Platform

from verta import Client

client = Client(
        #   host="app.verta.ai",
        #   email="user@verta.ai",
        #   dev_key="a765b2de-786d-466c-b2d8-thiye06f80d5",
        )

In [ ]:
# Create/Get a Verta registered model

registered_model = client.get_or_create_registered_model(
    "EfficientNet",
    desc="Family of EfficientNet models for ImageNet Classification - 1000 Classes",
    labels=["CV", "Classification", "Neural Net"],
)

In [ ]:
from verta.environment import Python

# uncommment the below if you want to load it from a saved model (h5 or tf saved)
# import tf
# model = tf.keras.models.load_model("<file_to_model.h5> or <path_to_tf_saved_model>")

model_version_v1 = registered_model.create_standard_model_from_keras(
    model, # The loaded model object, can be the one trained in the same file or loaded from keras load_model function
    environment=Python(requirements=[ # Add the required libraries for the model to run
    "tensorflow"
    ]), 
    name="V2M-tf", # Name to identify the version in the model versions tab
)

And That's it. You should now be able to see your Model, on your Catalog.